In [1]:
import Random

In [173]:
# open boundary (chapter 4)

function mcs_open(vehicles, velocities, max_velocity, min_velocity, p)
    """
    Perform 1 MCS. Velocity update happens in parallel for every car. 
    """
    
    update_velocities_open(vehicles, velocities, max_velocity, min_velocity, p)
    
    print_state(vehicles, velocities)
    
    update_positions_open(vehicles, velocities)
    
end


function update_velocities_open(vehicles, velocities, max_velocity, min_velocity, p)
    """
    Update velocities of vehicles in open system. Performed in-place.
    At the beginning next_vehicle points to empty space, later to vehicle in front.
    1. If next_vehicle is not in range of current vehicle's move, try to accelerate.
       If next_vehicle is in range of move, slow down.
    2. Random slow down event
    """
    
    length = size(vehicles)[2]
    
    # Holds index at which collision occurs. At the beginning we take impossible index.
    next_vehicle = length + max_velocity + 1
    
    for i = length:-1:1
        # if vehicle exists on position
        if vehicles[1,i] != 0
            
            # if no collision
            if next_vehicle > i + velocities[1,i]
                
                accelerate(velocities, i, next_vehicle, max_velocity)
                
                slow_down_random(velocities, i, p)
                
            else
                
                slow_down_car(velocities, i, next_vehicle)
                
                slow_down_random(velocities, i, p)
                
            end
            
            next_vehicle = i
        end
    end
end


function update_positions_open(vehicles, velocities)
    """
    Update positions in open system given current positions and velocities. Performed in-place.
    If vehicle at one of the last 6 places, delete it.
    If vehicle has speed 0, leave it.
    If not, move vehicle to new index = (its index + its velocity).
    Crashes are taken care of in update_velocity, no need to check them here
    """
    length = size(vehicles)[2]
    for i = length:-1:1
        if vehicles[1,i] != 0 && velocities[1,i] != 0
            
            # Check if move out of bounds
            if i > length - 6
                vehicles[1,i] = 0
                velocities[1,i] = 0
            else
                # Update new position in vehicles and velocities
                # Clear old position
                
                vehicles[1, (i + velocities[1,i])] = 1
                velocities[1, (i + velocities[1,i])] = velocities[1,i]
                vehicles[1,i] = 0
                velocities[1,i] = 0
            end
        end
    end
end


# try to accelerate
function accelerate(velocities, pos, next_vehicle, max_velocity)
    """
    If velocity is smaller than max_velocity chceck if collision occurs.
    If no collision => accelerate
    """
    if velocities[1,pos] < max_velocity
        if pos + velocities[1,pos] + 1 < next_vehicle
            velocities[1,pos] += 1
        end
    end
end


# slow down due to next car
function slow_down_car(velocities, pos, next_vehicle)
    """
    Slow down to land on position (next_vehicle - 1) after move.
    """
    velocities[1,pos] = next_vehicle - pos - 1
end


# slow down randomly
function slow_down_random(velocities, pos, p)
    """
    Random event of slowing down with probability p. 
    Velocity cannot drop below 0.
    """
    if Random.rand() < p
        if velocities[1,pos] > 0
            velocities[1,pos] -= 1
        end
    end
end


slow_down_random (generic function with 1 method)

In [174]:
# closed boundary (chapter 3)

function mcs(vehicles, velocities, max_velocity, min_velocity, p)
    """
    Perform 1 MCS. Velocity update happens in parallel for every car. 
    """
    
    update_velocities(vehicles, velocities, max_velocity, min_velocity, p)

    print_state(vehicles, velocities)
    
    update_positions(vehicles, velocities)
    
end


function update_velocities_closed(vehicles, velocities, max_velocity, min_velocity, p)
    """
    Update velocities of vehicles in closed system. Performed in-place.
    At the beginning next_vehicle points to the leftmost vehicle (because of periodic boundary), 
    but in conception (length + lefmost position). Thanks to that, velocity updates are simple.
    Later it points to vehicle in front of currenly checked one.
    
    1. If next_vehicle is not in range of current vehicle's move, try to accelerate.
       If next_vehicle is in range of move, slow down.
    2. Random slow down event
    """
    
    length = size(vehicles)[2]
    next_vehicle = 0
    
    for i = 1:length
        if vehicles[i] != 0
            next_vehicle = i + length
            break
        end
    end
    
    for i = length:-1:1
        # if vehicle exists on position
        if vehicles[1,i] != 0
            
            # if no collision
            if next_vehicle > i + velocities[1,i]
                
                accelerate(velocities, i, next_vehicle, max_velocity)
                
                slow_down_random(velocities, i, p)
                
            else
                
                slow_down_car(velocities, i, next_vehicle)
                
                slow_down_random(velocities, i, p) 
            end
            
            next_vehicle = i
        end
    end
end


function update_positions_closed(vehicles, velocities)
    """
    Update positions in closed system given current positions and velocities. Performed in-place.
    If vehicle has speed 0, leave it.
    If not, move vehicle to new index = (its index + its velocity).
    If vehicle moves out of the system, move it to the left side.
    
    Crashes are taken care of in update_velocity, no need to check them here
    """
    
    length = size(vehicles)[2]
    
    # If rightmost vehicle goes through periodic boundary, avoid updating it second time by saving it's new position
    # At the beginning avoid_pos holds value out of bounds
    avoid_pos = length+10
    
    for i = length:-1:1
        if (vehicles[1,i] != 0) && (velocities[1,i] != 0) && (i != avoid_pos)
            
            # Check if vehicle crosses boundary
            if (i + velocities[1,i]) > length
                
                # If yes, update modulus of position (so 81 becomes 1)
                new_pos = (i + velocities[1,i])%length
                
                vehicles[1, new_pos] = 1
                velocities[1, new_pos] = velocities[1,i]
                avoid_pos = new_pos
                vehicles[1,i] = 0
                velocities[1,i] = 0
                
            else
                vehicles[1, (i + velocities[1,i])] = 1
                velocities[1, (i + velocities[1,i])] = velocities[1,i]
                vehicles[1,i] = 0
                velocities[1,i] = 0
            end
            
        end
    end
end


update_positions_closed (generic function with 1 method)

In [157]:
function print_state(vehicles, velocities)
    length = size(vehicles)[2]
    str = ""
    for i = 1:length
        if vehicles[i] == 0
            str = str * "."
        else
            str = str * string(velocities[i])
        end
    end
    println(str)
end

print_state (generic function with 1 method)

Init

In [162]:
length = 80
p = 0.3
max_velocity = 5 
min_velocity = 0

0

In [172]:
# open boundary
velocities = fill(0, (1,length))
vehicles = fill(0, (1,length))

no_steps = 40

for step = 1:no_steps
    if vehicles[1,1] == 0
        vehicles[1,1] = 1
        velocities[1,1] = 0
    end
    mcs_open(vehicles, velocities, max_velocity, min_velocity, p)
    # if no vehicle on the leftmost position, create one with velocity 0

end

0...............................................................................
0...............................................................................
1...............................................................................
02..............................................................................
1..3............................................................................
02....4.........................................................................
1..2......5.....................................................................
01...3.........5................................................................
0.2.....4...........5...........................................................
0...3.......5............4......................................................
0......3.........5...........5..................................................
0.........4...........5...........5.............................................
1.............5............4

In [175]:
# closed boundary

velocities = fill(0, (1,length))
vehicles = fill(0, (1,length))

no_cars = 20
no_steps = 40

# place no_cars cars with velocity 0 on random spots
vehicles[1, 1:no_cars] .= 1
Random.shuffle!(vehicles)

for step = 1:no_steps
    mcs_closed(vehicles, velocities, max_velocity, min_velocity, p)
end

..1...0..1.01..........1.1......1...0.........0.00.1.........1..1...1.0.0.1...1.
...2..1...00.2..........1.1......2..1.........1.01..1.........2..2...01.1..2...1
2....0.2..01...3.........0.1.......0.2.........01.2..2..........1..1.0.1.2...2..
..1..1...00.2.....3......1..1......1...3.......0.2..2..3.........2..01..2..3...2
.0.2..1..00...2......3....1..2......2.....3....1...1..3...4........01.2...3...1.
.1...1.0.01.....3.......2..2...3......3......1..2...2....3....4....1.2..3....1.0
..1...01.1.1.......4......1..3....4......4....2...3...2.....4.....1.1..3...2..01
1..1..1.0.0.2..........3...2....4.....5......2..2....2..3.......2..0.2....2..01.
.2..2..01.1...3...........2..3......5......3...2..3....3...4......01...2....01.0
...1..00.0.2.....4..........2...4........4....2..2...4....3....2..0.2....2..1.00
....1.01.1...2.......5........3.....5........2..2..3.....3...3...01...3....1.000
.....01.0.2....3..........4......4.......5.....1..2...4.....3...01.2.....1..0001
2....1.01...3.....4.........